In [2]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [4]:
URL = "https://boulanger.com/c/nav-filtre/smartphone-telephone-portable?_etat_produit~neuf"

In [5]:
session = create_session()
url_list = create_url_list(URL, 11) #11
pages = read_pages(url_list=url_list, session=session)

Page 1 -> Successfull Extraction.
Page 2 -> Successfull Extraction.
Page 3 -> Successfull Extraction.
Page 4 -> Successfull Extraction.
Page 5 -> Successfull Extraction.
Page 6 -> Successfull Extraction.
Page 7 -> Successfull Extraction.
Page 8 -> Successfull Extraction.
Page 9 -> Successfull Extraction.
Page 10 -> Successfull Extraction.


Elapsed time for read_pages function: 39.687 seconds.

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

In [ ]:
all_urls

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [8]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [9]:
smartphones_json = to_json(smartphone_list)

In [10]:
file_path = "data/smartphones_2.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [1]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *
from itertools import chain

In [2]:
df_brut = pl.read_json("data/smartphones_2.json")

### Pipeline

Sur `535` extractions, il y a `487` téléphones valides.

In [3]:
df = NordStream(df_brut)

In [5]:
from pathlib import Path

root = Path(".").resolve()
data_folder = root / "data"
path_file = data_folder / "df_clean_2.csv"

In [3]:
df.write_csv(path_file, separator=";")

NameError: name 'df' is not defined

In [2]:
import polars as pl
import plotly.express as px

In [3]:
df = pl.read_parquet("data/sfa_results_app.parquet")

In [32]:
df.select("price", "ram", "storage", "screen_size", "ppi").corr()

price,ram,storage,screen_size,ppi
f64,f64,f64,f64,f64
1.0,0.49409,0.675877,0.120186,0.584822
0.49409,1.0,0.535435,0.381094,0.489469
0.675877,0.535435,1.0,0.209919,0.360114
0.120186,0.381094,0.209919,1.0,-0.027821
0.584822,0.489469,0.360114,-0.027821,1.0


In [37]:
df.select(
    pl.col(
        "price", "ram", "storage", "screen_size",
        "das_head", "das_limbs", "das_chest", "ppi",
        "battery", "repairability_index"
    )
).describe().transpose(include_header=True)

column,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8
str,str,str,str,str,str,str,str,str,str
"""describe""","""count""","""null_count""","""mean""","""std""","""min""","""25%""","""50%""","""75%""","""max"""
"""price""","""432.0""","""0.0""","""663.1971990740…","""484.5014346060…","""79.0""","""249.0""","""499.0""","""999.0""","""1999.0"""
"""ram""","""432.0""","""0.0""","""7.037037037037…","""2.922592771678…","""2.0""","""4.0""","""8.0""","""8.0""","""16.0"""
"""storage""","""432.0""","""0.0""","""228.4814814814…","""188.7042878181…","""8.0""","""128.0""","""128.0""","""256.0""","""1000.0"""
"""screen_size""","""432.0""","""0.0""","""6.490092592592…","""0.343801581394…","""4.7""","""6.3""","""6.6""","""6.7""","""7.92"""
"""das_head""","""432.0""","""0.0""","""0.864337962962…","""0.196778643332…","""0.26""","""0.76""","""0.978""","""0.99""","""1.301"""
"""das_limbs""","""432.0""","""0.0""","""2.607499999999…","""0.510668798547…","""0.86""","""2.25""","""2.596""","""2.98""","""3.8"""
"""das_chest""","""432.0""","""0.0""","""1.114560185185…","""0.236313166883…","""0.189""","""0.98""","""0.998""","""1.29""","""1.73"""
"""ppi""","""432.0""","""0.0""","""413.9076893594…","""71.53102891961…","""261.8711904413…","""396.4935063676…","""411.2196037398…","""457.2543463119…","""722.2646459286…"


In [9]:
df.select(pl.col("price").std())

price
f64
484.501435


In [30]:
ctx = pl.SQLContext()

In [41]:
ctx.register("df", df).execute(
    "SELECT ram FROM df WHERE model = 'iPhone 14 Plus' AND color = 'Jaune' "
    ).collect().item()

4.0

In [31]:
df.filter(pl.col("efficiency") < 0.8).select(pl.col("model").count()).item()

85

In [1]:
df.select(pl.corr("ram", "storage"))

NameError: name 'df' is not defined

In [78]:
fig_ram = px.box(df_ram, x="ram_fmt", y="price", points="all")

In [77]:
for trace in fig_ram.data:
    print(trace.type)

box


In [ ]:
import polars as pl
from pathlib import Path
from script.polars_gazprom import NordStream

root = Path(".").resolve()
data_folder = root / "data"
phones = data_folder / "smartphones.json"

df_brut = pl.read_json(phones)

df = NordStream(df_brut)

In [3]:
from IPython.display import IFrame

<iframe frameborder="0" height="480" width="640" allowFullScreen webkitallowfullscreen="true" mozallowfullscreen="true" onmousewheel="" src="https://www.mecabricks.com/en/player/1w2r09DVv8W"></iframe>

In [12]:
IFrame(
    "https://www.mecabricks.com/en/player/1w2r09DVv8W",
    width=600,
    height=300,
    frameborder =0
)